In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chat_models import ChatOpenAI
import os
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from openai import OpenAI
import streamlit as st
import tiktoken

In [12]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1)

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load_and_split()

In [14]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [15]:
# chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

In [16]:
model_name ="jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [17]:
docsearch = Chroma.from_documents(texts, hf)
retriever = docsearch.as_retriever()

In [21]:
contextualize_q_system_prompt = (
      "당신은 한국 노동법에 대한 전문 지식을 가진 AI 어시스턴트입니다. "
      "답변은 사용자가 질문한 언어와 같은 언어를 사용하세요."
      "다음 질문에 대해 주어진 문맥을 사용하여 상세하고 정확한 답변을 제공해주세요. "
      "관련 문맥 부분을 참조하여 답변을 작성하세요.\n\n"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [22]:
system_prompt = (
    "당신은 한국 노동법에 대한 전문 지식을 가진 AI 어시스턴트입니다. "
    "You MUST Answer in Korean."
    "다음 질문에 대해 주어진 문맥을 사용하여 상세하고 정확한 답변을 제공해주세요. "
    "관련 문맥 부분을 참조하여 답변을 작성하세요.\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [24]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [27]:
conversational_rag_chain.invoke(
    {"input": "유럽으로 여행가고 싶은데 장기휴가 최대 얼마나 낼 수 있어?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'근로기준법에 따르면, 1년간 80퍼센트 이상 출근한 근로자에게는 15일의 유급휴가를 주어야 하며, 3년 이상 계속하여 근로한 근로자에게는 추가로 2년에 1일씩 가산하여 최대 25일의 유급휴가를 주어야 합니다. 따라서, 유럽으로 여행을 가려면 최대 25일의 휴가를 낼 수 있습니다.'